<a href="https://colab.research.google.com/github/rukmals/crowd-monitoring-system-model-development/blob/main/UCF_QNRF_Data_SetUp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from tensorflow.keras.preprocessing import image

import scipy
from scipy.io import loadmat
import glob
import h5py
import time
from sklearn.externals.joblib import Parallel, delayed
from torchvision import datasets, transforms

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [3]:
test_setpath = "/content/drive/MyDrive/Model_Test_Data/UCF-QNRF/UCF-QNRF_ECCV18/Test"

In [4]:
def gaussian_filter_density(gt):
    print(gt.shape)
    density = np.zeros(gt.shape, dtype=np.float32)
    gt_count = np.count_nonzero(gt)
    if gt_count == 0:
        return density
    #print(gt_count)
    pts = np.array(list(zip(np.nonzero(gt)[1], np.nonzero(gt)[0])))
    leafsize = 2048
    # build kdtree
    pts_copy = pts.copy()
    tree = scipy.spatial.KDTree(pts_copy, leafsize=leafsize)
    # query kdtree
    distances, locations = tree.query(pts, k=4)
    print('generate density...')
    for i, pt in enumerate(pts):
        pt2d = np.zeros(gt.shape, dtype=np.float32)
        pt2d[pt[1], pt[0]] = 1.
        if gt_count > 1:
            sigma = (distances[i][1] + distances[i][2] + distances[i][3]) * 0.1
        else:
            sigma = np.average(np.array(gt.shape)) / 2. / 2.  # case: 1 point
        density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
    print('done.')
    return density

In [26]:
def generate_density_map(img_path):
    img_path = "/content/drive/MyDrive/Model_Test_Data/UCF-QNRF/UCF-QNRF_ECCV18/Test/"+ img_path
    #print(img_path)
    mat_path = img_path.replace('.jpg', '.mat').replace('.','_ann.')
    #print(mat_path)
    mat = scipy.io.loadmat(mat_path)
    imgfile = image.load_img(img_path)
    img = image.img_to_array(imgfile)
    k = np.zeros((img.shape[0], img.shape[1]))
    gt = mat["annPoints"]
    #print(mat)
    #print(gt)
    for i in range(0, len(gt)):
        if int(gt[i][1]) < img.shape[0] and int(gt[i][0]) < img.shape[1]:
            k[int(gt[i][1]), int(gt[i][0])] = 1
    k = gaussian_filter_density(k)
    output_path = img_path.replace('.jpg', '.h5').replace('Test/','ground-truth-h5/')
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)
    print("output", output_path)
    with h5py.File(output_path, 'w') as hf:
        hf['density'] = k
    return img_path

In [22]:
test_list = os.listdir(test_setpath)
print(test_list)

['img_0183.jpg', 'img_0045.jpg', 'img_0287_ann.mat', 'img_0242_ann.mat', 'img_0152_ann.mat', 'img_0011.jpg', 'img_0078_ann.mat', 'img_0128_ann.mat', 'img_0221_ann.mat', 'img_0078.jpg', 'img_0135_ann.mat', 'img_0249_ann.mat', 'img_0243_ann.mat', 'img_0088.jpg', 'img_0198_ann.mat', 'img_0257_ann.mat', 'img_0053.jpg', 'img_0220_ann.mat', 'img_0069.jpg', 'img_0174_ann.mat', 'img_0315.jpg', 'img_0111.jpg', 'img_0068.jpg', 'img_0264_ann.mat', 'img_0143.jpg', 'img_0167.jpg', 'img_0244.jpg', 'img_0151_ann.mat', 'img_0097_ann.mat', 'img_0297.jpg', 'img_0093.jpg', 'img_0048.jpg', 'img_0044.jpg', 'img_0260_ann.mat', 'img_0188_ann.mat', 'img_0251.jpg', 'img_0256.jpg', 'img_0329.jpg', 'img_0267_ann.mat', 'img_0223.jpg', 'img_0144.jpg', 'img_0082.jpg', 'img_0318.jpg', 'img_0149_ann.mat', 'img_0096.jpg', 'img_0215_ann.mat', 'img_0086_ann.mat', 'img_0134.jpg', 'img_0122_ann.mat', 'img_0108.jpg', 'img_0201_ann.mat', 'img_0125.jpg', 'img_0148.jpg', 'img_0157.jpg', 'img_0247.jpg', 'img_0138_ann.mat', 'im

In [ ]:
generate_density_map('img_0001.jpg')

(1875, 2500)
generate density...


In [24]:
img_test_list = []
for item in test_list:
  if '.jpg' in item:
    img_test_list.append(item)
print(len(img_test_list))

334
